
*** Data Science Capstone Project Presentation***
---
### In fulfillment of Simplilearn Master Data Science Certification course

## Project_name: Healthcare - NIDDK (National Institute of Diabetes and Digestive and Kidney Diseases)
---
Presenter_:_***Samuel_Y._Ntsua***

### Trainer and Mentor : **TBD**

** install packages : <br>
** remotezip : for query and downloand zip folder from a url <br>
** plotly, seaborn :  interactive graphs <br>
** cufflinks : connects plotly with pandas to create graphs and charts of dataframes directly <br>
** textblob : process textual data<br>
** missingno : visualize missing data<br>
** scikit-learn : missing data treatment and model evaluation<br>
** pandas-profiling: detailed EDA.<br>
** In addition tho these, we will be installing other libraries down the road, as and when they are needed.

In [ ]:
# %pip install remotezip cufflinks plotly textblob seaborn missingno  # Comented out because this is needed only once.
# %pip install pandas_profiling[notebook]
# %conda install -c conda-forge scikit-learn pandas-profiling

In [ ]:
# Checking content of zip folder so I can determine a strategy to load files

In [ ]:
%lsmagic

In [ ]:
# Ignore harmless warnings
import warnings
warnings.filterwarnings('ignore')
# Imports
import os, pandas as pd, numpy as np, seaborn as sns, plotly.express as px, cufflinks as cf, matplotlib.pyplot as plt, missingno as missno
#os.listdir("Project2")

** Check the content of the ziped folder located on github

In [ ]:
# from remotezip import RemoteZip
# with RemoteZip('https://github.com/Simplilearn-Edu/Data-Science-Capstone-Projects/raw/master/Project_2.zip') as hczip :
#     for hcfiles in hczip.infolist():
#         print(hcfiles.filename)

**Now that we see the contents, we can grab the specific file we need for this project.

In [ ]:
# with RemoteZip('https://github.com/Simplilearn-Edu/Data-Science-Capstone-Projects/raw/master/Project_2.zip') as hczip :    
#    hczip.extract('Project 2/Healthcare - Diabetes/health care diabetes.csv')

**The file is now downloaded to my local machine at 'Project 2/Healthcare - Diabetes/health care diabetes.csv'<br>
** We can now load it with pandas<br>

In [ ]:
hc_df = pd.read_csv('Project 2/Healthcare - Diabetes/health care diabetes.csv')

## Exploratory Data Analysis

** Descriptive analysis and data understanding

In [ ]:
hc_df.shape

In [ ]:
type(hc_df)

In [ ]:
hc_df.dtypes

** Visual exploration and checking for missing data<br>
The project instruction indicates that a value of 0 in <br>
Glucose, BloodPressure, SkinThickness, Insulin, BMI are actually missing values<br>
So let's go ahead and set the 0s to np.nan <br>

In [ ]:
# hc_df.columns = hc_df.columns.map(str.lower) # all column names to lowercase
zcol = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
#hc_df=hc_df[hc_df[zcol].astype(float)]
#hc_df=hc_df.astype({'Glucose':float, 'BloodPressure':float, 'SkinThickness':float, 'Insulin':float, 'BMI':float})

In [ ]:
hc_df

In [ ]:
hc_df[hc_df[zcol]==0]=np.nan

In [ ]:
hc_df.isnull().sum()

** The same information as above, but now in percentge<br>

In [ ]:
round(100*(hc_df.isnull().sum() / len(hc_df)))

In [ ]:
# %history -g -f cmd_hist.py

In [ ]:
# %lsmagic
%matplotlib inline

** To visually see how missing value in one column is related to another column, 
*** let's plot the heatmap of the missingness.

In [ ]:
sns.heatmap(hc_df.isnull(),cbar=False)

** Furthermore, we can use missno's matrix to highlight places in each column where data is missing.

In [ ]:
missno.matrix(hc_df,figsize=(10,6))

** Dropping missing a missing value from a column leads to dropping all other valid <br>
** values in the row corresponding to the missing value. <br>
** Dropping valid data will then lead to bias in the results.<br>
** To avoid such problem, we need to examine how much good data will be thrown out when a targeted missing data is dropped.<br>
** A correlation metrix between the missing variables can tell us the how one missing value is related to other non-missing values.<br>
** Below, we see the missno heatmap of missingness correlated matrix.<br>

In [ ]:
missno.heatmap(hc_df,figsize=(10,6))

** Likewise, a dendogram also show how one missing value in one column is related to non-missing value in another column.

In [ ]:
missno.dendrogram(hc_df,figsize=(10,6))


** From the graphs above we can observe that:<br>
** (1) `Insuline` and `SkinThinckness` are correlated (corr coef =0.7)<br>
** (2) There are more missing data in `Insulin` and `SkinThickness`, but <br>
** each time `Insulin` is missing `SkinThickness` is also missing, however, not the other way round.<br>
** In theory, given this correlation, the absence of one of `Insulin` or `SkinThickness` will not affect the ability to predict<br>
** the `Outcome` varibale. That is, one of the correlated variable can be expressed in termes of the other.<br> 
** But if we were to drop one, further analysis is needed to determine the one that is not a `principal component` contributing<br> 
** variation to `Outcome`. Furthermore, when two variables are correlated, it do not necessarily mean one of then is useless. <br>
** In fact ,in this dataset, the measures of `Insulin` or `SkinThickness` were observed naturally as they occured on the patients.<br> 
** These observations suggest that dropping these missing values can lead to loosing data pont that would otherwise <br>
** contribute to the accuracy of our analysis results. <br>
** So for now we continue our analysis without dropping any variable.<br> 

** Going forward, we need to figure out how to handle the missing values then.<br> 

** Do we fill them with mean values? Probably not. Why?<br> 
** Although replacing almost half of the values with the mean value is not going the affect the mean of that same variable, <br>
** it however reduces it's standard error, and so affecting its relationship with other variables in the dataset.<br> 
** Doing so will likely tilt the imputed mean towards the observed mean.<br> 

** Replacing missing values with the mean constitute a quick fix that will get me the project<br>
** completed quickly, but it comes with flawed prediction capabilities.<br>

** As such, I would not opt for that option.<br>
** What else can be done without loosing predictive capability of the data? <br>

** Now let's look at two other treatment methods of missing values: <br>
** multiple imputation (`mi`) and maximum likelihood estimate (`mle`). <br>
** `maximum likelihood estimate` `mle`: <br>
** `mle` takes the row on which data is missing, then compare the non-missing values of that row to other non-missing <br>
** value in the same column (within variable), then determins the closet of the set of non-missing values (the likelyhood),<br>
** and finally look up the corresponding value in the missing value's column to replace the actual missing value. <br>
** Another way to view this is, if two subject have the same values of parameter except that one is missing for a subject, <br>
** it is logical to replace the missing value with the corresponding parameter of the other subject. <br>
** Problem with `mle` treatment of missing value <br>
** `mle` does not impute data. <br>
** Given the description of the method above, it is clear that the replacement of the missing value is `linear` in nature, <br>
** and therefore `mle` applies to linear models only. <br>
** `multiple imputation` `mi`: <br>
** As the name suggests, `mi` imputes multiple times, that is, it takes multiple and different samples (of same size) <br>
** from the original data (nonparametric bootstrap), compute an estimator $\hat{X_{i}}$ of the missing value from each sample, <br>
** then based on the assumption that, the missing value we are trying to figure out follows the same distribution as  $\hat{X_{i}}$, <br>
** we compute an estimate $\bar{x}$ of the missing value. <br>
** Statistical software like Stata, SAS, SPSS and R implement various computation methods of `mi`. <br>
** In Python I am going to use scikit-learn implimentation (IterativeImputer), even though it is still experimental as of today. <br>

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
imp = IterativeImputer(random_state=100)
imp.fit(hc_df)

In [ ]:
imputed_hc_df = pd.DataFrame(imp.transform(hc_df), columns=hc_df.columns)

In [ ]:
round(100*(imputed_hc_df.isnull().sum() / len(imputed_hc_df)))

### Now that we have full dataset, we can steam forward  with more data exploration.<br>

** Let's look at the Mean, std, min, max and quantiles of `before imputation`, for both `Outcome` ==0 and `Outcome` ==1

In [ ]:
hc_df.describe().T

The above output of `mean` and  `max` implies that there are some extremes values in the dataset.<br>
For instance, `Pregnancies` has a max value of 17.<br>
This suggests someone in the dataset was pregnant 17 times! Is this realistc?<br>
`Insulin` also seems to have an extreme case.<br>
Let's look at the historgram of the variable to see if there are extreme cases to worry about.<br>

In [ ]:
plt.figure(figsize=(10, 10))

for i, c in enumerate(hc_df.columns):
    plt.subplot(5,2,i+1)
    sns.histplot(hc_df[c])
    plt.title('Distribution plot for field:' + c)
    plt.xlabel('')
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

Looking at the graph above, two cases of `Pregnancies` and `Insulin` need explanation: <br>
`Pregnancies` : Even though it seems unusual for humans to be pregnant 17 times, the the graph shows no gap between<br>
the max value of 17 and the rest of the group. This suggest it is not an outlier case.<br><br>
`Insulin`: there are a few observations in the same region as the max value of 846.<br>
Even though these values are extremely high, the `75 percentile` shows that 75% of the data has value less than 190<br>
Furthermore, the `mean` of `155` and `standard deviation` of `118` shows that the bulk of the data are within normal range.<br>
Thus, the values of `Insulin` in the region of 800 will not significantly affect the overall statistical validity of that variable.<br>
`SkinThickness` and `MBI` seem a little skewed, but not to the level that warrant correction before analysis. 

** Mean, std, min, max and quantiles of `before imputation`, for `Outcome==0`

In [ ]:
hc_df[hc_df['Outcome']==0].describe().T

** Mean, std, min, max and quantiles of `before imputation`, for `Outcome==1`

In [ ]:
hc_df[hc_df['Outcome']==1].describe().T

`Differences` of (Mean, std, min, max and quantiles) `before and after inputation`:<br>
* We compute and compare the differences for `Outcome==0` and `Outcome==1` <br>

** Computation for `Outcome==0`

In [ ]:
round(imputed_hc_df[imputed_hc_df['Outcome']==0].describe().T -  hc_df[hc_df['Outcome']==0].describe().T ,4)

** Comment for `Outcome==0`:<br>
* `count`: We successfully augmented the count for `SkinThickness`, `Insulin`, `BloodPressure`, `Glucose` and `MBI` <br>
as shown in the first column of the table above<br>
* `mean`,`std`, `min`, `max` and `quantiles`: Given the magnitude of range (`max` - `min`) shown in the `before inputation`, <br>
`744-14 for Insulin, for instance`, it appears the before/after difference for `mean`, `std`, `min`, `max` and `quantiles` <br>
are quite small.<br>

** Computation for `Outcome==1`

In [ ]:
round(imputed_hc_df[imputed_hc_df['Outcome']==1].describe().T -  hc_df[hc_df['Outcome']==1].describe().T ,4)

** Comment for `Outcome==1`:<br>
* `count`: We successfully augmented the count for `SkinThickness`, `Insulin`, `BloodPressure`, `Glucose` and `MBI` <br>
as shown in the first column of the table above<br>
* `mean`,`std`, `min`, `max` and `quantiles`: Given the magnitude of range (`max` - `min`) shown in the `before inputation`, <br>
`846-14 for Insulin, for instance`, it appears the before/after difference for `mean`, `std`, `min`, `max` and `quantiles` <br>
are quite small.<br>

In conclusion, we can say the imputation has successfully improved the data without loosing any case.<br>

** But that does not mean that a thoughfully imputed dataset garantees a representative analysis result.<br>
** If there are significantly more of a particular outcome that others, the analysis results will be skeweb<br>
** if the dataset is not balance or a probability weight is not introduced. <br>
** Let's see how balanced is the data after imputation.<br>

In [ ]:
print("Count for Outcome types : \n", imputed_hc_df['Outcome'].value_counts())

In [ ]:
sns.countplot(imputed_hc_df['Outcome'])
plt.xlabel('Outcome types')
plt.ylabel('Count of Outcome types')

The graph above show that about `2/3` of `Outcome` is `0` or `negative outcome`.<br>
This implies that if we were to predict a 0 (negative outcome), we would have achieve an accuracy of 75% with the imbalanced data.<br>

The data is imbalanced. Data imbalance can be addressed during (a) the analysis and interpretation <br>
of results, including resampling methods or (b) at the model performance and evaluation metrics level, including chaging the metric and penalizing the algorithm computing the metric.<br>




# For this project, we are going to utilize the resampling method to balance the dataset.
# ! pip install imbalanced-learn

In [ ]:
from sklearn.utils import resample
outcome_maj = imputed_hc_df[imputed_hc_df.Outcome==0]
outcome_min = imputed_hc_df[imputed_hc_df.Outcome==1]
upsample_outcome_min = resample(outcome_min, replace =True, n_samples = outcome_maj.shape[0], random_state= 9876)
#We can now put the balanced and imputed dataset together
bal_imp_hc_df = pd.concat([outcome_maj, upsample_outcome_min])
# Count 

In [ ]:
print('Count of the values of Outcome :\n',bal_imp_hc_df.Outcome.value_counts())

In [ ]:
sns.countplot(bal_imp_hc_df.Outcome)
plt.xlabel('Outcome types')
plt.ylabel('Count of Outcome types')

After oversampling the minority class, we now have `Outcome` values to be 50/50.

In [ ]:
print('The shape of the data after oversampling \n {}'.format(bal_imp_hc_df.shape))

In [ ]:
bal_imp_hc_df.info()

In [ ]:
sns.pairplot(bal_imp_hc_df, hue='Outcome', diag_kind='kde', kind = 'reg')
plt.title('Pair scatter plots between variables')

Frem the pairplot above, it appears there is a slight correlation between : <br>
a-) `Glucose` and `Insulin` <br>
b-) `SkinThickness` and `BMI` <br>

Let's graph the correlation matrix too to confirm the above observation.<br>

In [ ]:
# Correlation heatmap
plt.figure(dpi=120)
sns.heatmap(bal_imp_hc_df.corr(), annot=True,cmap='Greens')

In [ ]:
bal_imp_hc_df.corr()['Outcome']

As expected, the correlation matrix show the highest correlated pairs are : <br>
`Glucose` and `Insulin`  0.68 <br>
`SkinThickness` and `BMI` 0.7 <br>
The lest correlated pair are `Age` and `DiabetesPedigreeFunction` 0.0057.<br>

Taking the factors individually, the most correlated to `Outcome` is `Glucose` (no surprise there) <br>
followed by `Insulin`, then `BMI`, then `Age`, then `Pregnancies` and `Skinthickness` and <br>
the least correlated with `Outcome` is `DiabetesPedigreeFunction`.<br>
This ranking also gives us a rought indication of the princial components that, together, would bring the most variations <br>
to `Outcome`, thus the best prediction power.

## Data Modeling
Our data has labels, that is, `Outcome` variable.<br>
We can therefore use a supervised learning algorithms to try to understand the relationship between: <br>
(a) 'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age' on one side and <br>
(b) `Outcome` on the other side. <br>
Given the variables in `(a)`, we are trying to determine if a patient has diabetes `(Outcome ==1)` or not `(Outcome==0)`<br>
This is a classification problem.<br>
Now, which of the classification algorithms will give us the best prediction model?<br>
The figure below from scikit-lean website shows us the path decide.<br>

![Choosing the right ML estimator](https://scikit-learn.org/stable/_static/ml_map.png)

To figure out which classification model will yield the best prediction of `Outcome`,<br>
we can train test a few classification model then evaluate the performance then choose the highest performing model.<br>

# Strategies for model building and selection.<br>
We will attempt several models to see their performance using Area Under the Curve score (AUC).<br>
We will then choose the best performing model (or combination of model).<br>
<br>
** Following scikit-learn's diagram above, I will model the data with the following 5 `estimators` :<br>
1) Support Vector machines (SVM) <br>
2) KNeighbors Classifier <br>
3) three (3) Ensemble Classifiers (RandomForestClassifier, ExtraTreesClassifier and XGBoost)<br>


 In the first round of model selection, I will look at the `performance report card`  of the `estimators`:<br>
 `precision`, `recall` and `f1-score`.
 The top 3 estimators will then be further evaluated by `parameter tuning` for their individual best performance on the data <br>
 Then they will be ranked by their `AUC score`.

** Split,Train and evaluate

In [ ]:
## Model algorights to test on data

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
import xgboost as xgb # !pip install xgboost (if not already installed)
from xgboost import XGBClassifier

# Methods for model Selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score,GridSearchCV
# Methods for model performance evaluation
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, mean_squared_error

In [ ]:
X, y = bal_imp_hc_df.drop('Outcome',axis=1), bal_imp_hc_df['Outcome']
X_train, X_test, y_train, y_test = train_test_split( X , y, test_size=0.3, random_state=9876)

In [ ]:
print(f'Shape of the training and testing splits: \n X_train ==> {X_train.shape} \n X_test ==> {X_test.shape} \n y_train ==> {y_train.shape}  \n y_test ==> {y_test.shape}')

## Performance report card:Precision, recall  and f1-score

##### Performance report card:  Support Vector Machines

In [ ]:
model = SVC()
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print(model)
print(f'Train performance \n ====================================================' )
print(classification_report(y_train, y_train_hat))

print(f'Test performance \n ====================================================' )
print(classification_report(y_test, y_test_hat))

print(f'Roc_auc score \n ====================================================' )
print(roc_auc_score(y_test, y_test_hat))

#####  Performance report card: K-Neighbors

In [ ]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print(model)
print(f'Train performance \n ====================================================' )
print(classification_report(y_train, y_train_hat))

print(f'Test performance \n ====================================================' )
print(classification_report(y_test, y_test_hat))

print(f'Roc_auc score \n ====================================================' )
print(roc_auc_score(y_test, y_test_hat))

##### Performance report card: Random Forest

In [ ]:
model = RandomForestClassifier(n_jobs=-1,random_state=9876)
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)
print(model)
print(f'Train performance \n ====================================================' )
print(classification_report(y_train, y_train_hat))

print(f'Test performance \n ====================================================' )
print(classification_report(y_test, y_test_hat))

print(f'Roc_auc score \n ====================================================' )
print(roc_auc_score(y_test, y_test_hat))

##### Performance report card: Extra Trees

In [ ]:
model = ExtraTreesClassifier(random_state=9876)
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)
print(model)
print(f'Train performance \n ====================================================' )
print(classification_report(y_train, y_train_hat))

print(f'Test performance \n ====================================================' )
print(classification_report(y_test, y_test_hat))

print(f'Roc_auc score \n ====================================================' )
print(roc_auc_score(y_test, y_test_hat))

##### Performance report card: XGBoost

In [ ]:
model = XGBClassifier(random_state=9876)
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print(model)
print(f'Train performance \n ====================================================' )
print(classification_report(y_train, y_train_hat))

print(f'Test performance \n ====================================================' )
print(classification_report(y_test, y_test_hat))

print(f'Roc_auc score \n ====================================================' )
print(roc_auc_score(y_test, y_test_hat))

# Parameter tuning: Model optimization.
Out of the 5 algorithms tested above, I will retain the best 3 AUC score.<br>
Random forest, XGBoost and ExtraTrees<br>
I will tune their parameters to increase AUC score, by uing cross-validation method `GridSearchCV`.

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

##### Parameter tuning: Random forest

In [ ]:
params = {
    'n_estimators': [100, 200, 400],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [1,2,4,5],
    'min_samples_leaf': [1,2,4,5],
    'max_leaf_nodes': [4,10,20,50,None]
}

rf_gs = GridSearchCV(RandomForestClassifier(n_jobs=-1,random_state=9876), params, n_jobs=-1, cv=KFold(n_splits=3), scoring='roc_auc')
rf_gs.fit(X_train, y_train)
y_pred = rf_gs.predict(X_test)
print(f'Best score for Random Forest:, {rf_gs.best_score_} \n Best parameters found for Random Forest: {rf_gs.best_params_}')

##### AUC score: Randon Forest 

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
rf_roc_auc = roc_auc_score(y_test, rf_gs.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, rf_gs.predict_proba(X_test)[:,1]) 

plt.figure()
plt.plot(fpr, tpr, label='Random Forest (area = %0.2f)' % rf_roc_auc) 
plt.plot([0, 1], [0, 1],'--')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Random Forest ROC curve with best parameters') 
plt.legend(loc="lower right")
plt.savefig('RF_ROC')
print('Area Under Curve: %.3f' % rf_roc_auc)
plt.show()

##### Parameter tuning: XGBoost

In [ ]:
params = {
    'n_estimators': [100, 200, 400],
    'learning_rate': [0.01,0.05,0.1],
    'booster': ['gbtree', 'gblinear'],
    'gamma': [0, 0.5, 1],
    'reg_alpha': [0, 0.5, 1],
    'base_score': [0.2, 0.5, 1]
}

xgb_gs = GridSearchCV(XGBClassifier(n_jobs=-1, random_state=9876), params, n_jobs=-1, cv=KFold(n_splits=3), scoring='roc_auc')
xgb_gs.fit(X_train, y_train)
print(f'Best score for XGBoost:, {xgb_gs.best_score_} \n Best parameters found for XGBoost: {xgb_gs.best_params_}')

##### AUC score: XGBoost 

In [ ]:
xgb_roc_auc = roc_auc_score(y_test, xgb_gs.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, xgb_gs.predict_proba(X_test)[:,1]) 

plt.figure()
plt.plot(fpr, tpr, label='XGBoost (area = %0.2f)' % xgb_roc_auc) 
plt.plot([0, 1], [0, 1],'--')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('XGBoost ROC curve with best parameters') 
plt.legend(loc="lower right")
plt.savefig('XGB_ROC')
print('Area Under Curve: %.3f' % xgb_roc_auc)
plt.show()

##### Parameter tuning: Extra Trees

In [ ]:
params = {
    'n_estimators': [100, 200, 400],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [1,2,4,6],
    'min_samples_leaf': [1,2,4,6],
    'max_leaf_nodes': [4,10,20,40,None]
}

et_gs = GridSearchCV(ExtraTreesClassifier(n_jobs=-1, random_state=9876), params, n_jobs=-1, cv=KFold(n_splits=3), scoring='roc_auc')
et_gs.fit(X_train, y_train)
print(f'Best score for ExtraTrees:, {et_gs.best_score_} \n Best parameters found for ExtraTrees: {et_gs.best_params_}')

##### AUC score: Extra Trees

In [ ]:
et_roc_auc = roc_auc_score(y_test, et_gs.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, et_gs.predict_proba(X_test)[:,1]) 

plt.figure()
plt.plot(fpr, tpr, label='Extra Trees (area = %0.2f)' % et_roc_auc) 
plt.plot([0, 1], [0, 1],'--')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Extra Trees ROC curve with best parameters') 
plt.legend(loc="lower right")
plt.savefig('ET_ROC')
print('Area Under Curve: %.3f' % et_roc_auc)
plt.show()

# Conclusion: Model selection<br>
#### We can see that Extra Trees has the best ability to predict with this dataset.<br>
#### With a AUC score of 86%, Extra Trees perfomed better than Random Forest 85%, then XGBoost with 82%.<br>
#### KNN didn't even make ti to the top 3 <br>